
# Capstone Project: "Transforming Neighborhoods by Predicting Crime Outcomes in the UK
# Name: Marina Mnoyan
# Contact: marina.mnoyan@gmail.com
# Date: July 4, 2022
# Project Purpose:
Help policymakers to identify and address the factors that influence whether crimes are solved. The main dataset was downloaded from the UK Police Department website. Additional datasets were acquired from various UK Government sources in order to provide neighborhood-level characteristics. 


# Project Framework & Deliverables

    1. Data Acquisition, pre-processing with MS Excel & Git Bash       MS Excel, Git Bash (details in the ReadMe text file)         2. Main dataset Wrangling, Processing & Cleaning                   Jupyter Notebook 1     <== current notebook
    3. Additional dataset Merging, Wrangling, Processing & Cleaning    Jupyter Notebook 2
    4. Exploratory Data Analysis & Visualizations                      Jupyter Notebook 3
                                                                       Tableau File
    5. Feature Engineering                                             Jupyter Notebook 4
    6. Modeling & Evaluation                                           Jupyter Notebook 5
    7. Findings & Recommendations                                      PDF Business Report
                                                                       PDF Presentation
    8. Additional Information including links to original data         ReadMe text file

# Jupyter Notebook 1 of 5: Main dataset Wrangling, Processing & Cleaning

Steps:

[1) Data Reading and Basic Checks ](#1) 

[2) Data Cleaning and Processing](#2)

[3) CSV Output](#3)

<a id='1'></a>
## 1) Data Reading & Basic Checks

First, we will import basic packages that will help with this deliverable:

In [1]:
# Import basic data science packages
import numpy as np
import pandas as pd

# Filter warnings
import warnings
warnings.filterwarnings('ignore')

Then we need to make CSV files readable for Jupyter Notebook. Please note that some pre-processing was done with Git Bash to combine separate downloaded files into one CSV file (details in the ReadMe text file).

In [2]:
# Importing the CSV file
df_crime = pd.read_csv('data/combined-csv-files-crime.csv')

Now, let's perform some basic checks to familiarize ourselves with the data.

First, let's take a peak at what it looks like:

In [3]:
# Show the first five rows of the dataset
df_crime.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,f7f56767d4e4c717f500a1ccdb8754151a33598f98e209...,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-0.837531,51.827141,On or near Carlton Close,E01017707,Aylesbury Vale 012C,Violence and sexual offences,Status update unavailable,NaN
1,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.512153,51.412941,On or near Heathfield Close,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
2,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.513718,51.429407,On or near A4174,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
3,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.509126,51.416137,On or near St Francis Road,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN,NaN
4,a13ac1b5a26d154227c2d6c0cf3e3db1f1d39b54f93f7d...,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.511927,51.409435,On or near Harlech Close,E01014399,Bath and North East Somerset 001A,Burglary,Status update unavailable,NaN


We can see that we are dealing with missing data in some of the columns, however no duplication has been noticed. We'll need to check the percentage of missing values to see which columns we can keep, and what kind of analysis we can perform.

There are also columns which may not be useful for the scope of our analysis which we may consider dropping later.

Here are descriptions of the columns:

* **Crime ID:**                 A unique ID given to each crime in the neighborhood
* **Month:**                    A Month/Year combination of when the crime was committed
* **Report by:**                Police Jurisdiction that reported the Crime
* **Falls within:**             Police Jurisdiction where the crime occurred
* **Longitude:**                Longitude where crime occurred
* **Latitude:**                 Latitude where crime occurred
* **Location:**                 Intersection or particular location of where the crime occurred
* **LSOA code:**                Neighborhood code
* **LSOA name:**                Neighborhood name
* **Crime type:**               Type of crime committed
* **Last outcome category:**    Outcome of the investigation
* **Context:**                  Additional information

Next, let's check how big our dataset is:

In [4]:
# Returns the number of rows and columns
df_crime.shape

(19003667, 12)

Look like the dataset has over 1.9MM rows and 12 columns. It is a quite large dataset which may need a lot of computational power, so we may need to come up with a way to reduce it.

Now let's look at how many columns have missing data:

In [5]:
# Shows the # of rows with missing data
df_crime.isnull().sum()

Crime ID                  4635413
Month                           1
Reported by                     1
Falls within                    1
Longitude                  355458
Latitude                   355458
Location                        1
LSOA code                  841365
LSOA name                  841365
Crime type                      1
Last outcome category     4932023
Context                  19002084
dtype: int64

Looks like quite a lot of the data is missing but let's look at it in terms of percentages:

In [6]:
print("The percentage of null rows: ", df_crime.isnull().sum() / len(df_crime) * 100)

The percentage of null rows:  Crime ID                 24.392203
Month                     0.000005
Reported by               0.000005
Falls within              0.000005
Longitude                 1.870471
Latitude                  1.870471
Location                  0.000005
LSOA code                 4.427382
LSOA name                 4.427382
Crime type                0.000005
Last outcome category    25.953007
Context                  99.991670
dtype: float64


Unfortunately, it looks like the `Context` column is missing 99% of its data so we cannot use NLP to do sentiment analysis. Additional columns with large proportions of missing values are `Last Outcome Category` and `Crime ID`. 

We will not really need Crime ID for our analysis so can safely drop the column.  However, Last Outcome Category is our dependent variable so we will drop the observations with missing values rather than try to impute it as it will mess up with our analysis and modeling.

Given the size of the dataset and relatively small proportion of missing data in other columns, we will just drop the rows with missing values.

Now, let's see how many duplicated rows we have (we can already see that columns are not duplicated):

In [7]:
# Checking the number of duplicated rows
df_crime.duplicated().sum()

1749722

In [8]:
# Printing the percentage of duplicated rows
print("The percentage of duplicated rows: ", df_crime.duplicated().sum() / len(df_crime) * 100)

The percentage of duplicated rows:  9.207286151667466


It looks like over 9% of rows are duplicated. Before dropping them, it would be good to check whether these are the ones that have the Crime ID (meaning they are true duplicates) and don't have Crime ID (meaning that they may not be duplicates, and just similar crimes occurring):

In [9]:
# Selecting duplicate rows except first
# Occurrence based on all columns
duplicate = df_crime[df_crime.duplicated()]
 
print("Duplicate Rows:")
 
# Print the resultant Dataframe
duplicate.head(50)

Duplicate Rows:


,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
14,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.494715,51.419948,On or near Somerdale Road,E01014400,Bath and North East Somerset 001B,Anti-social behaviour,NaN,NaN
17,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.498613,51.416002,On or near High Street,E01014400,Bath and North East Somerset 001B,Anti-social behaviour,NaN,NaN
18,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.498613,51.416002,On or near High Street,E01014400,Bath and North East Somerset 001B,Anti-social behaviour,NaN,NaN
20,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.498613,51.416002,On or near High Street,E01014400,Bath and North East Somerset 001B,Anti-social behaviour,NaN,NaN
21,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.498613,51.416002,On or near High Street,E01014400,Bath and North East Somerset 001B,Anti-social behaviour,NaN,NaN
22,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.498613,51.416002,On or near High Street,E01014400,Bath and North East Somerset 001B,Anti-social behaviour,NaN,NaN
23,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.498613,51.416002,On or near High Street,E01014400,Bath and North East Somerset 001B,Anti-social behaviour,NaN,NaN
57,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.507544,51.405741,On or near Newlands Road,E01014402,Bath and North East Somerset 002A,Anti-social behaviour,NaN,NaN
84,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.502878,51.406282,On or near Dunster Road,E01014404,Bath and North East Somerset 002C,Anti-social behaviour,NaN,NaN
85,NaN,2019-04,Avon and Somerset Constabulary,Avon and Somerset Constabulary,-2.502878,51.406282,On or near Dunster Road,E01014404,Bath and North East Somerset 002C,Anti-social behaviour,NaN,NaN


Looking at the duplicated rows, it looks like many of them are missing the Crime ID. Additionally, the Committed Crimes are generally related to Public Order and Anti-social behavior. Since these crimes tend to involve more than one individual, we will not drop the duplicates and will treat them as unique entries.

Lastly, let's check our column types: 

In [10]:
# Checking column dtype
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19003667 entries, 0 to 19003666
Data columns (total 12 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   Crime ID               object
 1   Month                  object
 2   Reported by            object
 3   Falls within           object
 4   Longitude              object
 5   Latitude               object
 6   Location               object
 7   LSOA code              object
 8   LSOA name              object
 9   Crime type             object
 10  Last outcome category  object
 11  Context                object
dtypes: object(12)
memory usage: 1.7+ GB


Looks like there is an opportunity to separate the month and the year and cast them as integers, as well as change longitude and latitude to floats.

<a id='2'></a>
## 2) Data Cleaning & Processing

First of all, let's start looking at the columns. First let's see if there is a big difference between `Reported by` and `Falls within` columns, i.e. if we need both of them or just one:

In [11]:
# Checking the percent of 'sameness': 
(df_crime['Reported by'] == df_crime['Falls within']).mean()

0.9999166476659478

Looks like the columns have a very significant overlap.

At this point we can make the decision to drop the following columns: 

* `Crime ID`: It has a lot of missing values and will likely not be useful in our future analysis
* `Context`: It has 99% missing values
* `Location`: While this is great data for a more granular analysis, additional demographic datasets are available on the neighborhood (LSOA) level only, so we will drop this column for now. It can be used in future analyses.
* `Falls within`: Since there is a 99.99% overlap between this column and `Reported  by` column, we can safely drop it

In [12]:
# Dropping the columns
df_crime.drop(columns=['Crime ID', 'Context', 'Falls within', 'Location'], inplace=True)

We will also rename the `Reported by` column into `Police Jurisdiction` for ease of understanding

In [13]:
# Renaming
df_crime.rename(columns = {'Reported by':'Police_Jurisdiction'}, inplace = True)

Let's do a sanity check to make sure everything is ok:

In [14]:
# Viewing the first 5 rows
df_crime.head()

,Month,Police_Jurisdiction,Longitude,Latitude,LSOA code,LSOA name,Crime type,Last outcome category
0,2019-04,Avon and Somerset Constabulary,-0.837531,51.827141,E01017707,Aylesbury Vale 012C,Violence and sexual offences,Status update unavailable
1,2019-04,Avon and Somerset Constabulary,-2.512153,51.412941,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN
2,2019-04,Avon and Somerset Constabulary,-2.513718,51.429407,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN
3,2019-04,Avon and Somerset Constabulary,-2.509126,51.416137,E01014399,Bath and North East Somerset 001A,Anti-social behaviour,NaN
4,2019-04,Avon and Somerset Constabulary,-2.511927,51.409435,E01014399,Bath and North East Somerset 001A,Burglary,Status update unavailable


Everything looks ok.

Now that we have our columns, let's drop the null values. This will also help reduce the size of the file:

In [15]:
# Dropping null values
df_crime.dropna(inplace=True)

Now, we can separate the current `Month` column into separate `Year` and `Month` columns:

In [16]:
# Extracting the year portion (before the hyphen) into the Year column
df_crime['Year'] = df_crime['Month'].str.split('-', expand=True)[0]

In [17]:
# Extracting the month portion (before the hyphen) into the Year column
df_crime['Month'] = df_crime['Month'].str.split('-', expand=True)[1]

Checking that this worked:

In [18]:
# Looking at first five rows
df_crime.head()

,Month,Police_Jurisdiction,Longitude,Latitude,LSOA code,LSOA name,Crime type,Last outcome category,Year
0,04,Avon and Somerset Constabulary,-0.837531,51.827141,E01017707,Aylesbury Vale 012C,Violence and sexual offences,Status update unavailable,2019
4,04,Avon and Somerset Constabulary,-2.511927,51.409435,E01014399,Bath and North East Somerset 001A,Burglary,Status update unavailable,2019
5,04,Avon and Somerset Constabulary,-2.515072,51.419357,E01014399,Bath and North East Somerset 001A,Burglary,Status update unavailable,2019
6,04,Avon and Somerset Constabulary,-2.516919,51.423683,E01014399,Bath and North East Somerset 001A,Public order,Status update unavailable,2019
7,04,Avon and Somerset Constabulary,-2.515072,51.419357,E01014399,Bath and North East Somerset 001A,Vehicle crime,Status update unavailable,2019


This worked! There are some problems with the index but we can fix it in the end.

Let's check that we didn't introduce any null values by performing the split:

In [19]:
# Checking for the number of rows containing null values
df_crime.isna().sum()

Month                    1583
Police_Jurisdiction         0
Longitude                   0
Latitude                    0
LSOA code                   0
LSOA name                   0
Crime type                  0
Last outcome category       0
Year                        0
dtype: int64

Looks like there are some small missing values in the Month category, let's drop them:

In [20]:
# Droppping null rows
df_crime.dropna(inplace=True)

Now let's change the `Month` and `Year` columns into integers to help with future EDA:

In [21]:
# Casting Month as integer
df_crime['Month'] = df_crime['Month'].astype(int)

In [22]:
# Casting Year as integer
df_crime['Year'] = df_crime['Year'].astype(int)

Performing sanity check:

In [23]:
# Checking Dtype of each column
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13738495 entries, 0 to 19003665
Data columns (total 9 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   Month                  int32 
 1   Police_Jurisdiction    object
 2   Longitude              object
 3   Latitude               object
 4   LSOA code              object
 5   LSOA name              object
 6   Crime type             object
 7   Last outcome category  object
 8   Year                   int32 
dtypes: int32(2), object(7)
memory usage: 943.3+ MB


Success!

While we would have loved to continue working with the full database, unfortunately this takes too long to run and crashed the laptop repeatedly. With this, we will extract the latest full year (year=2021) of information to move to the next step of our project:

In [24]:
# Extracting all rows where Year is 2021
df_crime = df_crime[(df_crime['Year']==2021)]

With this we don't really need the `Year` column anymore:

In [25]:
# Dropping the year column
df_crime.drop(axis=1, columns='Year', inplace=True)

Now we can change the type of our geographic data from object to float:

In [26]:
# Changing Longitude into a float
df_crime["Longitude"] = pd.to_numeric(df_crime["Longitude"], downcast="float")

In [27]:
# Changing Latitude into a float
df_crime["Latitude"] = pd.to_numeric(df_crime["Latitude"], downcast="float")

Let's check that this worked!

In [28]:
# Checking the Dtype of each column
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4580733 entries, 11301432 to 17545961
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Month                  int32  
 1   Police_Jurisdiction    object 
 2   Longitude              float32
 3   Latitude               float32
 4   LSOA code              object 
 5   LSOA name              object 
 6   Crime type             object 
 7   Last outcome category  object 
dtypes: float32(2), int32(1), object(5)
memory usage: 262.1+ MB


Everything worked as planned.

<a id='1'></a>
## CSV Output

Prior to outputting the cleaned file as CSV, let's check where we are and reset the index:

In [29]:
# Looking at first 5 rows
df_crime.head()

,Month,Police_Jurisdiction,Longitude,Latitude,LSOA code,LSOA name,Crime type,Last outcome category
11301432,1,Avon and Somerset Constabulary,-2.515072,51.419357,E01014399,Bath and North East Somerset 001A,Burglary,Status update unavailable
11301433,1,Avon and Somerset Constabulary,-2.494870,51.422276,E01014399,Bath and North East Somerset 001A,Criminal damage and arson,Status update unavailable
11301434,1,Avon and Somerset Constabulary,-2.494870,51.422276,E01014399,Bath and North East Somerset 001A,Criminal damage and arson,Status update unavailable
11301435,1,Avon and Somerset Constabulary,-2.511927,51.409435,E01014399,Bath and North East Somerset 001A,Public order,Unable to prosecute suspect
11301436,1,Avon and Somerset Constabulary,-2.511761,51.409966,E01014399,Bath and North East Somerset 001A,Violence and sexual offences,Unable to prosecute suspect


In [30]:
df_crime.shape

(4580733, 8)

We ended up with ~4MM observations and 8 columns to move into the next step of our project. Now let's reset the index

In [31]:
# Resetting the index
df_crime.reset_index(drop=True)

,Month,Police_Jurisdiction,Longitude,Latitude,LSOA code,LSOA name,Crime type,Last outcome category
0,1,Avon and Somerset Constabulary,-2.515072,51.419357,E01014399,Bath and North East Somerset 001A,Burglary,Status update unavailable
1,1,Avon and Somerset Constabulary,-2.494870,51.422276,E01014399,Bath and North East Somerset 001A,Criminal damage and arson,Status update unavailable
2,1,Avon and Somerset Constabulary,-2.494870,51.422276,E01014399,Bath and North East Somerset 001A,Criminal damage and arson,Status update unavailable
3,1,Avon and Somerset Constabulary,-2.511927,51.409435,E01014399,Bath and North East Somerset 001A,Public order,Unable to prosecute suspect
4,1,Avon and Somerset Constabulary,-2.511761,51.409966,E01014399,Bath and North East Somerset 001A,Violence and sexual offences,Unable to prosecute suspect
...,...,...,...,...,...,...,...,...
4580728,12,Wiltshire Police,-1.751553,50.992847,E01031995,Wiltshire 062E,Shoplifting,Investigation complete; no suspect identified
4580729,12,Wiltshire Police,-1.743673,50.993011,E01031995,Wiltshire 062E,Vehicle crime,Investigation complete; no suspect identified
4580730,12,Wiltshire Police,-1.739677,50.994244,E01031995,Wiltshire 062E,Vehicle crime,Investigation complete; no suspect identified
4580731,12,Wiltshire Police,-1.757983,50.992241,E01031995,Wiltshire 062E,Violence and sexual offences,Unable to prosecute suspect


At this point we are ready to export the processed file into a CSV, to be picked up in Jupyter Notebook 2:

In [32]:
df_crime.to_csv(r'data\df_crime.csv', index=False)

### This is the end of Jupyter Notebook 1 (of 5). Please proceed to Jupyter Notebook 2.